In [5]:
import os
import urllib.request
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import subprocess
import sys

def install_package(package):
    """Installs a package using pip, trying 'python3' and then 'python'."""
    try:
        print("Attempting to install package using 'python3'...")
        subprocess.check_call(["python3", "-m", "pip", "install", package])
        print(f"Successfully installed {package} with 'python3'")
    except FileNotFoundError:
        print("'python3' not found. Trying 'python' instead...")
        try:
            subprocess.check_call(["python", "-m", "pip", "install", package])
            print(f"Successfully installed {package} with 'python'")
        except FileNotFoundError:
            print("Error: Neither 'python3' nor 'python' command was found in the environment's PATH.")
            raise
        except subprocess.CalledProcessError as e:
            print(f"Error installing {package} with 'python': {e}")
            raise
    except subprocess.CalledProcessError as e:
        print(f"Error installing {package} with 'python3': {e}")
        raise


def download_file(url, target_path):
    """Downloads a file from a URL and saves it locally."""
    try:
        with urllib.request.urlopen(url) as response, open(target_path, 'wb') as out_file:
            data = response.read()
            out_file.write(data)
        print(f"Successfully downloaded {os.path.basename(target_path)}")
    except Exception as e:
        print(f"Error downloading {url}: {e}")
        raise

def run_all_analyses_and_visualizations():
    """
    Sets up a DSSAT environment, runs multiple analysis scenarios,
    and generates all relevant visualizations.
    """
    # First, ensure DSSATTools is installed
    install_package('DSSATTools')
    
    # Now import from the newly installed package
    from DSSATTools import (
        crop,
        WeatherStation,
        SoilProfile,
        filex,
        DSSAT
    )
    from DSSATTools.filex import IrrigationEvent, FertilizerEvent

    # ===================================================================
    # SECTION 1: BASIC SETUP AND DATA ACQUISITION
    # ===================================================================
    print("\n--- SECTION 1: Initializing Workspace and Data ---")
    workspace = os.path.join(os.getcwd(), 'dssat_simulation_workspace')
    os.makedirs(workspace, exist_ok=True)
    print(f"DSSAT workspace created at: {workspace}")

    soil_path = os.path.join(workspace, 'SOIL.SOL')
    download_file('https://raw.githubusercontent.com/DSSAT/dssat-csm-data/develop/Soil/SOIL.SOL', soil_path)

    weather_dfs = []
    for year in ['80', '81']:
        filename = f'ITHY{year}01.WTH'
        url = f'https://raw.githubusercontent.com/DSSAT/dssat-csm-data/develop/Weather/{filename}'
        path = os.path.join(workspace, filename)
        download_file(url, path)
        df = pd.read_fwf(path, colspecs=[(0, 5), (6, 11), (12, 17), (18, 23), (24, 29)], 
                         names=['date', 'srad', 'tmax', 'tmin', 'rain'], skiprows=4)
        weather_dfs.append(df)

    df_weather = pd.concat(weather_dfs, ignore_index=True)
    df_weather.dropna(subset=['date'], inplace=True)
    df_weather['date'] = pd.to_datetime(df_weather['date'], format='%y%j')
    df_weather.drop_duplicates(subset='date', keep='first', inplace=True)
    df_weather.sort_values(by='date', inplace=True)

    weather_station = WeatherStation(insi='ITHY', lat=17.53, long=78.27, elev=0, tav=25.8, amp=11.8, table=df_weather)
    soil = SoilProfile.from_file('IBSG910085', soil_path)
    dssat_env = DSSAT(workspace)
    
    # FINAL FIX: Provide detailed initial conditions for all soil layers.
    # This is crucial for the model to correctly simulate and output the daily water balance.
    initial_conditions = filex.InitialConditions(
        pcr='SG', icdat=datetime(1980, 6, 16), icres=1300.0,
        table=pd.DataFrame([
            #icbl, sh2o,  snh4, sno3
            (5,    0.22,  2.5,  1.8),
            (15,   0.22,  2.5,  1.8),
            (30,   0.233, 3.0,  2.0),
            (45,   0.245, 3.5,  2.0),
            (60,   0.237, 3.3,  2.0),
            (90,   0.233, 2.0,  1.0),
            (120,  0.24,  1.0,  0.7),
            (150,  0.243, 1.0,  0.6),
            (172,  0.25,  1.0,  0.6),
        ], columns=['icbl', 'sh2o', 'snh4', 'sno3'])
    )
    
    planting = filex.Planting(pdate=datetime(1980, 6, 17), ppop=18, plds='R', plrs=45, pldp=5)
    
    # ===================================================================
    # SECTION 2: TREATMENT COMPARISON (Rainfed vs. Irrigated)
    # ===================================================================
    print("\n--- SECTION 2: Running Treatment Comparison (Rainfed vs. Irrigated) ---")
    
    field = filex.Field(id_field='ITHY0001', wsta=weather_station, id_soil=soil)
    outputs_config = filex.SCOutputs(grout='Y', waout='Y', sumry='Y', vbose='Y')
    
    # Run Rainfed Treatment
    simulation_controls_rainfed = filex.SimulationControls(
        general=filex.SCGeneral(sdate=datetime(1980, 6, 16)),
        options=filex.SCOptions(water='Y', nitro='Y', symbi='N'),
        management=filex.SCManagement(irrig='N', ferti='N', harvs='M'),
        outputs=outputs_config
    )
    rainfed_summary = dssat_env.run_treatment(
        field=field,
        cultivar=crop.Sorghum('IB0026'),
        planting=planting,
        initial_conditions=initial_conditions,
        simulation_controls=simulation_controls_rainfed
    )
    rainfed_growth = dssat_env.output_tables.get('PlantGro').copy()

    # Create and read irrigation schedule from a CSV file
    irrigation_csv_path = os.path.join(workspace, 'irrigation_schedule.csv')
    irrigation_data = """date,amount
1980-07-15,20
1980-08-15,25
1980-09-01,15
"""
    with open(irrigation_csv_path, 'w') as f:
        f.write(irrigation_data)
    print(f"Created sample irrigation schedule at: {irrigation_csv_path}")
    
    schedule_df = pd.read_csv(irrigation_csv_path, parse_dates=['date'])
    
    irrigation_events = [
        IrrigationEvent(idate=row['date'], irval=row['amount'], irop='IR001')
        for index, row in schedule_df.iterrows()
    ]

    irrigation = filex.Irrigation(efir=1, table=irrigation_events)

    # Run Irrigated Treatment using the schedule from the file
    simulation_controls_irrigated = filex.SimulationControls(
        general=filex.SCGeneral(sdate=datetime(1980, 6, 16)),
        options=filex.SCOptions(water='Y', nitro='Y', symbi='N'),
        management=filex.SCManagement(irrig='R', ferti='N', harvs='M'),
        outputs=outputs_config
    )
    irrigated_summary = dssat_env.run_treatment(
        field=field,
        cultivar=crop.Sorghum('IB0026'),
        planting=planting,
        initial_conditions=initial_conditions,
        simulation_controls=simulation_controls_irrigated,
        irrigation=irrigation
    )
    irrigated_growth = dssat_env.output_tables.get('PlantGro').copy()
    irrigated_water = dssat_env.output_tables.get('SoilWat').copy()

    # Create Comparison Plot with Harvest Day markers
    plt.figure(figsize=(10, 6))
    plt.plot(rainfed_growth.index, rainfed_growth['GWAD'], label='Rainfed')
    plt.plot(irrigated_growth.index, irrigated_growth['GWAD'], label='Irrigated', linestyle='--')
    
    pdate_dssattools = planting['pdate']
    planting_date = datetime(pdate_dssattools.year, pdate_dssattools.month, pdate_dssattools.day)
    
    rainfed_mat_dap = rainfed_summary['mat']
    irrigated_mat_dap = irrigated_summary['mat']
    rainfed_harvest_date = planting_date + timedelta(days=rainfed_mat_dap)
    irrigated_harvest_date = planting_date + timedelta(days=irrigated_mat_dap)
    
    plt.axvline(x=rainfed_harvest_date, color='red', linestyle=':', label=f'Rainfed Harvest ({rainfed_mat_dap} DAP)')
    plt.axvline(x=irrigated_harvest_date, color='blue', linestyle=':', label=f'Irrigated Harvest ({irrigated_mat_dap} DAP)')
    
    plt.title('Treatment Comparison: Grain Growth')
    plt.xlabel('Date')
    plt.ylabel('Grain Weight (kg/ha)')
    plt.legend()
    plt.grid(True)
    comp_path = os.path.join(workspace, 'treatment_comparison_growth.png')
    plt.savefig(comp_path)
    print(f"Saved treatment comparison plot to: {comp_path}")
    plt.close()
    
    # ===================================================================
    # SECTION 3: PARAMETER SENSITIVITY (Varying 'p1' coefficient)
    # ===================================================================
    print("\n--- SECTION 3: Running Parameter Sensitivity Analysis (P1 Coefficient) ---")
    
    p1_values = [350, 380, 410, 440, 470]
    sensitivity_results = []
    
    cultivar_sens = crop.Sorghum('IB0026')
    simulation_controls_sens = filex.SimulationControls(
        general=filex.SCGeneral(sdate=datetime(1980, 6, 16)),
        options=filex.SCOptions(water='Y', nitro='Y', symbi='N'),
        management=filex.SCManagement(irrig='N', ferti='N', harvs='M'),
        outputs=filex.SCOutputs(sumry='Y') 
    )
    
    for p1 in p1_values:
        cultivar_sens['p1'] = p1
        print(f"Running simulation for p1 = {p1}...")
        summary = dssat_env.run_treatment(
            field=field,
            cultivar=cultivar_sens,
            planting=planting,
            initial_conditions=initial_conditions,
            simulation_controls=simulation_controls_sens
        )
        summary['p1'] = p1
        sensitivity_results.append(summary)
        
    results_df = pd.DataFrame(sensitivity_results)

    # Create Sensitivity Plots
    plt.figure(figsize=(10, 6))
    plt.plot(results_df['p1'], results_df['harwt'], marker='o')
    plt.title('Sensitivity Analysis: Yield vs. P1 Coefficient')
    plt.xlabel('P1 Value (GDD)')
    plt.ylabel('Grain Weight at Harvest (kg/ha)')
    plt.grid(True)
    sens_yield_path = os.path.join(workspace, 'sensitivity_yield_vs_p1.png')
    plt.savefig(sens_yield_path)
    print(f"Saved sensitivity plot (Yield vs P1) to: {sens_yield_path}")
    plt.close()

    plt.figure(figsize=(10, 6))
    plt.plot(results_df['p1'], results_df['mat'], marker='o')
    plt.title('Sensitivity Analysis: Maturity vs. P1 Coefficient')
    plt.xlabel('P1 Value (GDD)')
    plt.ylabel('Days to Maturity (dap)')
    plt.grid(True)
    sens_mat_path = os.path.join(workspace, 'sensitivity_maturity_vs_p1.png')
    plt.savefig(sens_mat_path)
    print(f"Saved sensitivity plot (Maturity vs P1) to: {sens_mat_path}")
    plt.close()

    # ===================================================================
    # SECTION 4: DETAILED DAILY TIME-SERIES PLOTS
    # ===================================================================
    print("\n--- SECTION 4: Generating Detailed Daily Plots ---")

    # Re-use results from the irrigated run in Section 2
    plt.figure(figsize=(10, 6))
    plt.plot(irrigated_growth.index, irrigated_growth['LAID'])
    plt.title('Daily Leaf Area Index (Irrigated Treatment)')
    plt.xlabel('Date')
    plt.ylabel('Leaf Area Index (LAI)')
    plt.grid(True)
    plt.axvline(x=irrigated_harvest_date, color='red', linestyle='--', label=f'Harvest Day ({irrigated_mat_dap} DAP)')
    plt.legend()
    lai_path = os.path.join(workspace, 'daily_lai.png')
    plt.savefig(lai_path)
    print(f"Saved LAI plot to: {lai_path}")
    plt.close()
    
    # Plot for Daily Yield (Grain Growth Rate)
    irrigated_growth['daily_yield'] = irrigated_growth['GWAD'].diff().fillna(0).clip(lower=0)
    plt.figure(figsize=(10, 6))
    plt.plot(irrigated_growth['DAP'], irrigated_growth['daily_yield'])
    plt.title('Daily Yield (Grain Growth Rate)')
    plt.xlabel('Days After Planting (DAP)')
    plt.ylabel('Daily Increase in Grain Weight (kg/ha/day)')
    plt.grid(True)
    plt.axvline(x=irrigated_mat_dap, color='red', linestyle='--', label='Harvest Day')
    plt.legend()
    daily_yield_path = os.path.join(workspace, 'daily_yield.png')
    plt.savefig(daily_yield_path)
    print(f"Saved daily yield plot to: {daily_yield_path}")
    plt.close()

    # Plot for Total Accumulated Yield
    plt.figure(figsize=(10, 6))
    plt.plot(irrigated_growth['DAP'], irrigated_growth['GWAD'], label='Total Accumulated Grain')
    
    final_yield = irrigated_growth.loc[irrigated_growth.index == irrigated_harvest_date, 'GWAD'].values[0]
    
    plt.axvline(x=irrigated_mat_dap, color='red', linestyle='--', label=f'Harvest Day ({irrigated_mat_dap} DAP)')
    plt.scatter(irrigated_mat_dap, final_yield, color='red', zorder=5, s=100, label=f'Final Yield: {int(final_yield)} kg/ha')
    
    plt.title('Total Accumulated Grain Yield Over Time')
    plt.xlabel('Days After Planting (DAP)')
    plt.ylabel('Total Grain Weight (kg/ha)')
    plt.legend()
    plt.grid(True)
    cumulative_yield_path = os.path.join(workspace, 'cumulative_yield.png')
    plt.savefig(cumulative_yield_path)
    print(f"Saved cumulative yield plot to: {cumulative_yield_path}")
    plt.close()

    # Plot for Daily Water Input (Rain & Irrigation)
    df_weather_indexed = df_weather.set_index('date')
    plot_dates = pd.date_range(start=planting_date, end=irrigated_harvest_date + timedelta(days=5))
    df_water_plot = pd.DataFrame(index=plot_dates)

    df_water_plot = df_water_plot.join(df_weather_indexed['rain'])
    df_water_plot.rename(columns={'rain': 'RAIN'}, inplace=True)
    df_water_plot['RAIN'] = df_water_plot['RAIN'].fillna(0)
    
    irrigation_series = schedule_df.set_index('date')['amount']
    df_water_plot = df_water_plot.join(irrigation_series)
    df_water_plot.rename(columns={'amount': 'IRRIGATION'}, inplace=True)
    df_water_plot['IRRIGATION'] = df_water_plot['IRRIGATION'].fillna(0)

    plt.figure(figsize=(12, 6))
    plt.plot(df_water_plot.index, df_water_plot['RAIN'], label='Rain', color='blue')
    plt.bar(df_water_plot.index, df_water_plot['IRRIGATION'], label='Irrigation', color='purple', width=1.0)

    plt.title('Daily Water Input (Rain & Irrigation)')
    plt.xlabel('Date')
    plt.ylabel('Water (mm/day)')
    
    max_water = max(df_water_plot['RAIN'].max(), df_water_plot['IRRIGATION'].max())
    plt.ylim(-5, max_water * 1.1)
    
    plt.legend()
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    water_input_path = os.path.join(workspace, 'daily_water_input.png')
    plt.savefig(water_input_path)
    print(f"Saved daily water input plot to: {water_input_path}")
    plt.close()

    # ===================================================================
    # SECTION 5: PLANT AND SOIL WATER ANALYSIS PLOTS
    # ===================================================================
    print("\n--- SECTION 5: Generating Plant and Soil Water Analysis Plots ---")

    # Plot 1: Plant Water Stress
    plt.figure(figsize=(10, 6))
    plt.plot(irrigated_growth['DAP'], irrigated_growth['WSGD'], label='Water Stress on Growth')
    plt.title('Daily Plant Water Stress (Irrigated Treatment)')
    plt.xlabel('Days After Planting (DAP)')
    plt.ylabel('Water Stress Factor (0=No Stress, 1=Max Stress)')
    plt.grid(True)
    plt.legend()
    stress_path = os.path.join(workspace, 'plant_water_stress.png')
    plt.savefig(stress_path)
    print(f"Saved plant water stress plot to: {stress_path}")
    plt.close()

    # Plot 2: Soil Moisture vs. Thresholds
    if irrigated_water is not None and not irrigated_water.empty and 'SW1' in irrigated_water.columns:
        top_layer_soil = soil.table[0]
        slll = top_layer_soil['SLLL'] 
        sdul = top_layer_soil['SDUL']

        plt.figure(figsize=(12, 6))
        plt.axhline(y=sdul, color='blue', linestyle='--', label=f'Field Capacity (SDUL): {sdul:.3f}')
        plt.axhline(y=slll, color='red', linestyle='--', label=f'Wilting Point (SLLL): {slll:.3f}')
        
        plt.plot(irrigated_water.index, irrigated_water['SW1'], label='Actual Soil Moisture (Top Layer)')
        
        plt.title('Daily Soil Moisture vs. Ideal Thresholds (Top Layer)')
        plt.xlabel('Date')
        plt.ylabel('Volumetric Water Content ($cm^3/cm^3$)')
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        moisture_path = os.path.join(workspace, 'soil_moisture_level.png')
        plt.savefig(moisture_path)
        print(f"Saved soil moisture plot to: {moisture_path}")
        plt.close()
    else:
        print("\nWARNING: 'SW1' column not found in SoilWat.OUT. Soil moisture plot will not be generated.")


if __name__ == "__main__":
    run_all_analyses_and_visualizations()

Attempting to install package using 'python3'...

--- SECTION 1: Initializing Workspace and Data ---
DSSAT workspace created at: d:\Final-Year-Project\dssat_simulation_workspace
Successfully downloaded SOIL.SOL
Successfully downloaded ITHY8001.WTH
Successfully downloaded ITHY8101.WTH
d:\Final-Year-Project\dssat_simulation_workspace created.

--- SECTION 2: Running Treatment Comparison (Rainfed vs. Irrigated) ---

RUN    TRT FLO MAT TOPWT HARWT  RAIN  TIRR   CET  PESW  TNUP  TNLF   TSON TSOC
           dap dap kg/ha kg/ha    mm    mm    mm    mm kg/ha kg/ha  kg/ha t/ha
  1 SG   1  67 109 11134  3871   614     0   373    98    82    14  13382   95
Created sample irrigation schedule at: d:\Final-Year-Project\dssat_simulation_workspace\irrigation_schedule.csv

RUN    TRT FLO MAT TOPWT HARWT  RAIN  TIRR   CET  PESW  TNUP  TNLF   TSON TSOC
           dap dap kg/ha kg/ha    mm    mm    mm    mm kg/ha kg/ha  kg/ha t/ha
  1 SG   1  67 109 11020  3772   614    60   375    98    80    12  13382  